In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import PIL

import torchvision as tv
from torchvision import datasets
from torchvision import transforms
from torchvision.utils import save_image
from torchsummary import summary

from pushover import notify
from utils import makegif
from random import randint

from IPython.display import Image
from IPython.core.display import Image, display
import numpy as np

%load_ext autoreload
%autoreload 2

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [3]:
bs = 32 # batchsize 32

In [4]:
# Load Data

def print_gt_zero_elem(matrix):
    print(matrix[matrix > 0])
    
# def load_img(img_path):
#     img = PIL.Image.open(img_path)
# #     img = PIL.ImageMath.eval("int(img)", img=img)
# #     print(img)
# #     img = PIL.ImageOps.grayscale(img)
#     img = img.convert(mode="L")
#     npimg = np.array(img)
#     npimg = npimg/256.0
#     return npimg
# IMRANGE = 256 # uint8
# # dataset = datasets.ImageFolder(root='trainings/rolls_gray', transform=transforms.Compose([
# # #     transforms.Resize(64),
# #     transforms.ToTensor(), 
# # #     lambda x: (x * IMRANGE).type(torch.IntTensor),
# #     lambda x: x.type(torch.FloatTensor),
# # ]), loader=load_img)

# dataset = datasets.ImageFolder(root='trainings/rolls_gray', transform=transforms.Compose([
#     transforms.ToTensor(), 
#     lambda x: x.type(torch.FloatTensor),
# ]), loader=load_img)

def load_img(img_path):
    img = PIL.Image.open(img_path)
    img = img.convert(mode="L")
#     npimg = np.array(img)/256.0
    return img

IMRANGE = 256 # uint8

dataset = datasets.ImageFolder(root='trainings/rolls_gray', transform=transforms.Compose([
    transforms.ToTensor(), 
    lambda x: x > 0,
    lambda x: x.type(torch.FloatTensor),
]), loader=load_img)

dataloader = torch.utils.data.DataLoader(dataset, batch_size=bs, shuffle=True)
len(dataset.imgs), len(dataloader)

# size of input = 3 x 128 x 128

(2297, 72)

In [5]:
# Fixed input for debugging
fixed_x, _ = next(iter(dataloader))
save_image(fixed_x, 'outputs/real_image.png')

# Image('outputs/real_image.png')

In [6]:
print(dataset[0][0].shape)
sample_dat = dataset[0][0]
print_gt_zero_elem(sample_dat)
HSIZE = 2048 #9216 # 1024
ZDIM = 16

torch.Size([1, 128, 128])
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])


In [7]:
class Flatten(nn.Module):
    def forward(self, input):
#         print("flatten: ", input.shape)
        return input.view(input.size(0), -1)

In [8]:
class UnFlatten(nn.Module):
    def forward(self, input, size=HSIZE):
        return input.view(input.size(0), size, 1, 1)

In [9]:
class VAE(nn.Module):
    def __init__(self, image_channels=1, h_dim=HSIZE, z_dim=ZDIM):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(image_channels, 32, kernel_size=4, stride=2), # -> [32, 32, 31, 31] 63
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2), # -> [32, 64, 14, 14] 31
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2), # -> [32, 128, 6, 6] 14
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=4, stride=2), # -> [32, 256, 2, 2] 6
            nn.ReLU(), 
            nn.Conv2d(256, 512, kernel_size=4, stride=2), # -> Null -> [32, 512, 2, 2] 
            nn.ReLU(), 
            Flatten() # -> [32, 1024]  -> [32, 2048]
            # [32, a, b, c] -> [32, abc]
        )
        
        self.fc1 = nn.Linear(h_dim, z_dim)
        self.fc2 = nn.Linear(h_dim, z_dim)
        self.fc3 = nn.Linear(z_dim, h_dim)
        
        self.decoder = nn.Sequential(
            UnFlatten(), 
            nn.ConvTranspose2d(h_dim, 256 , kernel_size=5, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=5, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=5, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=6, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(32, image_channels, kernel_size=6, stride=2),
            nn.Sigmoid(),
        )
#         self.decoder = nn.Sequential(
#             UnFlatten(),
#             nn.ConvTranspose2d(h_dim, 128, kernel_size=5, stride=2),
#             nn.ReLU(),
#             nn.ConvTranspose2d(128, 64, kernel_size=5, stride=2),
#             nn.ReLU(),
#             nn.ConvTranspose2d(64, 32, kernel_size=6, stride=2),
#             nn.ReLU(),
#             nn.ConvTranspose2d(32, image_channels, kernel_size=6, stride=2),
#             nn.Sigmoid(),
#         )
        
    def reparameterize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        # return torch.normal(mu, std)
        esp = torch.randn(*mu.size())
        z = mu + std * esp
        return z
    
    def bottleneck(self, h):
        mu, logvar = self.fc1(h), self.fc2(h)
#         print("bottle: ",mu.shape, logvar.shape)
        z = self.reparameterize(mu, logvar)
        return z, mu, logvar

    def encode(self, x):
#         print("======== Encode ========", x.shape)
        h = self.encoder(x)
#         print("enc(x): ", h.shape)
        z, mu, logvar = self.bottleneck(h)
#         print("z.shape: ", z.shape)
        return z, mu, logvar

    def decode(self, z):
#         print("======== Decode ========", z.shape)
        z = self.fc3(z)
#         print("fc3(z).shape: ", z.shape)
        z = self.decoder(z)
#         print("decode(fc3(z)).shape: ", z.shape)
        return z

    def forward(self, x):
        z, mu, logvar = self.encode(x)
#         print(z.shape)
        z = self.decode(z)
#         print(z.shape, mu.shape, logvar.shape)
        return z, mu, logvar

In [10]:
image_channels = fixed_x.size(1)

In [11]:
vae = VAE(image_channels=image_channels).to(device)
# model.load_state_dict(torch.load('vae.torch', map_location='cpu'))

In [12]:
optimizer = torch.optim.Adam(vae.parameters(), lr=1e-3)

In [13]:
def loss_fn(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x, size_average=False)
#     BCE = F.mse_loss(recon_x, x, size_average=False)

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD, BCE, KLD

In [15]:
epochs = 100

In [16]:
model_name = "graybin_bce_d16"

In [17]:
for epoch in range(epochs):
    for idx, (images, _) in enumerate(dataloader):
        recon_images, mu, logvar = vae(images)
#         print(images.shape)
        
        comimg = torch.cat([images * 256.0, recon_images * 256.0])
        sample_filename = 'tmp/sample_comp_image.png'
        save_image(comimg.data.cpu(), sample_filename)
        
#         display(Image(sample_filename, width=300, unconfined=True))
        
        loss, bce, kld = loss_fn(recon_images, images, mu, logvar)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

#         to_print = "Epoch[{}/{}] Loss: {:.3f} {:.3f} {:.3f}".format(epoch+1, 
#                                 epochs, loss.data[0]/bs, bce.data[0]/bs, kld.data[0]/bs)
        to_print = "Epoch[{}/{}] Loss: {:.3f} {:.3f} {:.3f}".format(epoch+1, 
                                epochs, loss.data/bs, bce.data/bs, kld.data/bs)
    if epoch % 10 == 0 and epoch != 0:
        torch.save(vae.state_dict(), 'models/intermediate/cvae.{}-imgs_{}-epch_{}-{}'.format(model_name, len(dataset.imgs), epoch, epochs))
        
    print(to_print, loss.data, kld.data, bs)

# notify to android when finished training
notify(to_print, priority=1)

/usr/local/lib/python3.6/site-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Epoch[1/100] Loss: 1022.473 1022.422 0.050 tensor(32719.1230) tensor(1.6123) 32
Epoch[2/100] Loss: 981.394 981.290 0.104 tensor(31404.6074) tensor(3.3354) 32
Epoch[3/100] Loss: 867.052 866.868 0.183 tensor(27745.6562) tensor(5.8669) 32
Epoch[4/100] Loss: 1002.440 1002.264 0.175 tensor(32078.0684) tensor(5.6124) 32
Epoch[5/100] Loss: 894.700 894.469 0.232 tensor(28630.4082) tensor(7.4089) 32
Epoch[6/100] Loss: 958.333 958.117 0.216 tensor(30666.6582) tensor(6.9247) 32
Epoch[7/100] Loss: 774.296 774.063 0.233 tensor(24777.4707) tensor(7.4503) 32
Epoch[8/100] Loss: 741.276 741.032 0.243 tensor(23720.8223) tensor(7.7861) 32
Epoch[9/100] Loss: 731.365 731.101 0.264 tensor(23403.6699) tensor(8.4423) 32
Epoch[10/100] Loss: 709.043 708.793 0.250 tensor(22689.3750) tensor(8.0074) 32
Epoch[11/100] Loss: 665.015 664.748 0.267 tensor(21280.4668) tensor(8.5294) 32
Epoch[12/100] Loss: 609.271 609.038 0.232 tensor(19496.6582) tensor(7.4290) 32
Epoch[13/100] Loss: 531.731 531.469 0.262 tensor(17015.38

In [18]:
torch.save(vae.state_dict(), 'models/cvae.{}-imgs_{}-epch_{}'.format(model_name, len(dataset.imgs), epochs))